Git submission

In [1]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['Colab Notebooks', 'Flicker8k_Dataset', 'German', 'Image Processing', 'additional_homework', 'datasets', 'descriptions.txt', 'epam', 'homework_09', 'homework_10', 'output', 'project_files', 'test.pkl', 'text']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import keras
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model


tensorflow: 2.17.1
keras: 3.5.0


In [3]:
def extract_features(directory):

	model = VGG16()

	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

	print(model.summary())

	features = {}

	for name in listdir(directory):
		filename = directory + '/' + name
		image = load_img(filename, target_size=(224, 224))
		image = img_to_array(image)
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		image = preprocess_input(image)
		feature = model.predict(image, verbose=0)
		image_id = name.split('.')[0]
		features[image_id] = feature
		print('>%s' % name)
	return features

In [ ]:
directory = 'Flickr8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
#dump(features, open('features.pkl', 'wb'))

In [25]:
filename = 'text/Flickr8k.token.txt'

def load_doc(filename):

	file = open(filename, 'r')

	text = file.read()

	file.close()
	return text

doc = load_doc(filename)

In [26]:
def load_descriptions(doc):
	mapping = {}

	for line in doc.split('\n'):

		tokens = line.split()
		if len(line) < 2:
			continue

		image_id, image_desc = tokens[0], tokens[1:]

		image_id = image_id.split('.')[0]
		image_desc = ' '.join(image_desc)
		if image_id not in mapping:
			mapping[image_id] = list()
		mapping[image_id].append(image_desc)
	return mapping


descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
print(descriptions["1358089136_976e3d2e30"])

Loaded: 8092 
['A boy sand surfing down a hill', 'A man is attempting to surf down a hill made of sand on a sunny day .', 'A man is sliding down a huge sand dune on a sunny day .', 'A man is surfing down a hill of sand .', 'A young man in shorts and t-shirt is snowboarding under a bright blue sky .']


In [29]:
import re
from collections import Counter

def tokenize(text):

    return [s.lower() for s in re.split(r'\W+', text) if len(s) > 1]

freqs = Counter()

# Tokenize and count occurrences
all_tokens = []
for descs in descriptions.values():
    for sentence in descs:
        tokens = tokenize(sentence)
        all_tokens.extend(tokens)

word_freqs = Counter(all_tokens)
specials = ["<pad>", "<start>", "<end>", "<unk>"]
#remove all words that appear once
vocab = specials + [word for word, count in word_freqs.items() if count >= 2]

word_to_index = {word: idx for idx, word in enumerate(vocab)}

def encode_sentence(sentence, word_to_index):

    tokens = tokenize(sentence)
    encoded = [word_to_index.get("<start>")]
    for token in tokens:
        if token in word_to_index:
            encoded.append(word_to_index[token])
        else:
            encoded.append(word_to_index.get("<unk>"))
    encoded.append(word_to_index.get("<end>"))
    return encoded

encoded_descriptions = {}
for image_id, descs in descriptions.items():
    encoded_descriptions[image_id] = [encode_sentence(sentence, word_to_index) for sentence in descs]


print("Vocabulary:", word_to_index)
print("Encoded descriptions:", encoded_descriptions)

Vocabulary: {'<pad>': 0, '<start>': 1, '<end>': 2, '<unk>': 3, 'child': 4, 'in': 5, 'pink': 6, 'dress': 7, 'is': 8, 'climbing': 9, 'up': 10, 'set': 11, 'of': 12, 'stairs': 13, 'an': 14, 'way': 15, 'girl': 16, 'going': 17, 'into': 18, 'wooden': 19, 'building': 20, 'little': 21, 'playhouse': 22, 'the': 23, 'to': 24, 'her': 25, 'cabin': 26, 'black': 27, 'dog': 28, 'and': 29, 'spotted': 30, 'are': 31, 'fighting': 32, 'tri': 33, 'colored': 34, 'playing': 35, 'with': 36, 'each': 37, 'other': 38, 'on': 39, 'road': 40, 'white': 41, 'brown': 42, 'spots': 43, 'staring': 44, 'at': 45, 'street': 46, 'two': 47, 'dogs': 48, 'different': 49, 'breeds': 50, 'looking': 51, 'pavement': 52, 'moving': 53, 'toward': 54, 'covered': 55, 'paint': 56, 'sits': 57, 'front': 58, 'painted': 59, 'rainbow': 60, 'hands': 61, 'bowl': 62, 'sitting': 63, 'large': 64, 'small': 65, 'grass': 66, 'plays': 67, 'fingerpaints': 68, 'canvas': 69, 'it': 70, 'there': 71, 'pigtails': 72, 'painting': 73, 'young': 74, 'outside': 75, 

In [30]:
print(encoded_descriptions["1358089136_976e3d2e30"])
# # save descriptions to file, one per line
# def save_descriptions(descriptions, filename):
# 	lines = list()
# 	for key, desc_list in descriptions.items():
# 		for desc in desc_list:
# 			lines.append(key + ' ' + desc)
# 	data = '\n'.join(lines)
# 	file = open(filename, 'w')
# 	file.write(data)
# 	file.close()

# # save descriptions
# save_descriptions(descriptions, 'descriptions.txt')

[[1, 145, 277, 1582, 376, 339, 2], [1, 76, 8, 493, 24, 260, 376, 339, 1583, 12, 277, 39, 446, 447, 2], [1, 76, 8, 552, 376, 1418, 277, 1584, 39, 446, 447, 2], [1, 76, 8, 1582, 376, 339, 12, 277, 2], [1, 74, 76, 5, 480, 29, 159, 8, 1585, 255, 903, 160, 340, 2]]


In [20]:
from pickle import load
def load_photo_features(filename, dataset):
	all_features = load(open(filename, 'rb'))
	features = {k: all_features[k] for k in dataset}
	return features